# Gym Intro + Monte Carlo Learning

Originally from https://skettee.github.io/post/monte_carlo_learning/ (in Korean)

## Load Libraries and Extensions

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from IPython.display import display, clear_output, Pretty
import numpy as np
from time import sleep
from tqdm import tqdm_notebook as tqdm

import gym

## Frozen Lake Environment

In [3]:
ENV_NAME = 'FrozenLake-v0'
N_STEP = 100

In [4]:
env = gym.make(ENV_NAME, is_slippery=False)
state = env.reset()

In [5]:
world = env.render(mode='ansi')
Pretty(world)


SFFF
FHFH
FFFH
HFFG


In [6]:
for step in range(N_STEP):
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action)    
    state = next_state
    
    # updated world display
    world = env.render(mode='ansi')
    clear_output(wait=True)
    display(Pretty(world))
    sleep(0.5)
    
    if done: # an episode finished
        print("Episode finished after {} timesteps".format(step + 1))
        break

  (Right)
SFFF
FHFH
FFFH
HFFG


Episode finished after 2 timesteps


In [7]:
env.action_space

Discrete(4)

There are 4 actions in Fronzen Lake.

$A = \{0, 1, 2, 3\}$   

Num	| Action
----|----
0 |	Move Left
1 |	Move Down
2 |	Move Right
3 |	Move Up

In [8]:
env.observation_space

Discrete(16)

There are 16 states as follows:

$S = \{0, 1, \cdots , 15\}$   

$\begin{vmatrix}
0 & 1 & 2 & 3 \\
4 & 5 & 6 & 7 \\
8 & 9 & 10 & 11 \\
12 & 13 & 14 & 15
\end{vmatrix}$

For each state, there are actions possible associated with next state, reward, and done as follows:
    
`{action: [(probability, nextstate, reward, done)]}`  

For example, let's look at State 6 and 14:

In [9]:
env.P[6]

{0: [(1.0, 5, 0.0, True)],
 1: [(1.0, 10, 0.0, False)],
 2: [(1.0, 7, 0.0, True)],
 3: [(1.0, 2, 0.0, False)]}

In [10]:
env.P[14]

{0: [(1.0, 13, 0.0, False)],
 1: [(1.0, 14, 0.0, False)],
 2: [(1.0, 15, 1.0, True)],
 3: [(1.0, 10, 0.0, False)]}

## Monte Carlo Learning

$G_t = R_{t+1} + \gamma R_{t+2} + \cdots + = \sum_{k=0}^{\infty} \gamma^{k} R_{t+k+1}$

$q_{\pi}(s,a) = \mathbb E_{\pi} [G_t | S_t = s, A_t = a]$ 

$q_*(s,a) = \max_{\pi} q_{\pi}(s,a)$ 

$ \pi_*(s, a) = \begin{cases}
1 & \text{if } a= \text{argmax}_{a \in A} q_\star(s,a) \\
0 & \text{otherwise}
\end{cases} $

$\begin{align}
N(S_t, A_t) & \leftarrow N(S_t, A_t) + 1 \\
Q(S_t, A_t) & \leftarrow Q(S_t, A_t) + \dfrac{1}{N(S_t, A_t)} \left( G_t - Q(S_t, A_t) \right)  
\end{align}$

In [11]:
n_state = env.observation_space.n
n_action = env.action_space.n
n_episode = 5000
GAMMA = .6

In [12]:
Q_table = np.zeros((n_state, n_action))
N_table = np.zeros((n_state, n_action))
R_table = np.zeros((n_state, n_action))

for episode in tqdm(range(n_episode)):
    memory = []
    state = env.reset()
    
    for step in range(N_STEP):
        action = env.action_space.sample()
        memory.append((state, action))
        
        next_state, reward, done, info = env.step(action)
        R_table[state][action] = reward
        state = next_state
        
        if done:
            for i in range(len(memory)):
                G_t = 0
                gamma = GAMMA
                for j in range(i, len(memory)):
                    S_t, A_t = memory[j]
                    if i == j:
                        G_t += R_table[S_t][A_t]
                    else:
                        G_t += gamma * R_table[S_t][A_t]
                        gamma = gamma * gamma
                S_t, A_t = memory[i]
                N_table[S_t][A_t] += 1
                Q_table[S_t][A_t] += (G_t - Q_table[S_t][A_t]) / N_table[S_t][A_t]
            break

## Solution

In [13]:
state = env.reset()
done = False

world = env.render(mode='ansi')
display(Pretty(world))
sleep(.5)


SFFF
FHFH
FFFH
HFFG


In [14]:
while not done:
    action = np.argmax(Q_table[state])
    state, reward, done, info = env.step(action)
    
    world = env.render(mode='ansi')
    clear_output(wait=True)
    display(Pretty(world))
    sleep(.5)
    
    if done and state == 15:
        print('\nSuccess!')

  (Right)
SFFF
FHFH
FFFH
HFFG



Success!
